# STAC specification

The **SpatioTemporal Asset Catalog (STAC)** is an emerging open standard for geospatial data that aims to increase the interoperability of geospatial data, particularly satellite imagery. 
[Many major data archives](https://stacspec.org/en/about/datasets/) now follow the STAC specification.

In this lesson we'll be working with the [Microsoft's Planetary Computer (MPC)](https://planetarycomputer.microsoft.com) STAC API. 

## MPC Catalog 
First, load the necessary packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
from shapely.geometry import Polygon

from pystac_client import Client 

import planetary_computer

from IPython.display import Image 

### Access
We use the `Client` function from the `pystac_client` package to access the catalog:

In [2]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

The `modifier` parameter is needed to access the data in the MPC catalog.

### Catalog Exploration


In [3]:
# Explore catalogue metadata
print('Title:', catalog.title)
print('Description:', catalog.description)

Title: Microsoft Planetary Computer STAC API
Description: Searchable spatiotemporal metadata describing Earth science datasets hosted by the Microsoft Planetary Computer


We can access its collections by using the `get_collections` method:

In [4]:
catalog.get_collections()

<generator object Client.get_collections at 0x0000029736E7C480>

The output is a **generator**:
- a special kind of **lazy object** in python over which you can iterate over like a list
- items in a generator don't exist in memory until you explicitly iterate over them or convert them to a list
- allows for a more efficient memory allocation
- once the generator is iterated over completely, it cannot bereused unless it is recreated

In [5]:
# Get collections and print their names

collections = list(catalog.get_collections()) # Turn generator into list
print('Number of collections:', len(collections))
print('Collections IDs (first 10)')
for i in range(10):
    print('-', collections[i].id)

Number of collections: 126
Collections IDs (first 10)
- daymet-annual-pr
- daymet-daily-hi
- 3dep-seamless
- 3dep-lidar-dsm
- fia
- gridmet
- daymet-annual-na
- daymet-monthly-na
- daymet-annual-hi
- daymet-monthly-hi


## Collection

Select a single collection using the `get_child()` catalog method and the collection ID as the parameters

In [26]:
# Access NAIP collection
naip_collection = catalog.get_child('naip')
naip_collection

<CollectionClient id=naip>

## Catalog search

# Catalog search
narrow down the search within the `catalog` by specifying a time range, an arewas of interest, and the collection ID
 2 ways to define the area of interest:
 - a GeoJSON-type dictionary with the coords of the AOI bounding box
 - as a list `[xmin, ymin, xmax, ymax]` with the coords defining the four corners of the bounding box
 goal: retrieve NAIP scenes over Santa Barbara from 2018 to 2023

In [12]:
# NCEAS bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-119.70608227128903, 34.426300194372274],
            [-119.70608227128903, 34.42041139020533],
            [-119.6967885126002, 34.42041139020533],
            [-119.6967885126002, 34.426300194372274],
            [-119.70608227128903, 34.426300194372274]
        ]
    ],
}

# Specify tmeporal range of interest
time_range = "2018-01-01/2023-01-01"

# Catalog search 
search = catalog.search(
    collections = ['naip'], 
                   intersects = bbox, 
                   datetime=time_range)
search

To get the items found in the search using `item_collection()` method:


In [13]:
items = search.item_collection()
len(items)

3

In [14]:
items

In [15]:
# Get the first item in the catalog search
item = items[0]
type(item)

pystac.item.Item

The STAC item is the core object in STAC catalog

The item **does not contain the data itself** it has properties (metadata) and asserts (links yo the actual data)

In [16]:
# Print ITE ID and properties
print('ID', item.id)
item.properties

ID ca_m_3411935_sw_11_060_20220513


{'gsd': 0.6,
 'datetime': '2022-05-13T16:00:00Z',
 'naip:year': '2022',
 'proj:bbox': [246930.0, 3806808.0, 253260.0, 3814296.0],
 'providers': [{'url': 'https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/',
   'name': 'USDA Farm Service Agency',
   'roles': ['producer', 'licensor']}],
 'naip:state': 'ca',
 'proj:shape': [12480, 10550],
 'proj:centroid': {'lat': 34.40624, 'lon': -119.71877},
 'proj:transform': [0.6, 0.0, 246930.0, 0.0, -0.6, 3814296.0, 0.0, 0.0, 1.0],
 'proj:code': 'EPSG:26911'}

In [17]:
# Examine the item's assets
item.assets

{'image': <Asset href=https://naipeuwest.blob.core.windows.net/naip/v002/ca/2022/ca_060cm_2022/34119/m_3411935_sw_11_060_20220513.tif?st=2025-11-24T18%3A04%3A41Z&se=2025-11-25T18%3A49%3A41Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-11-23T16%3A24%3A35Z&ske=2025-11-30T16%3A24%3A35Z&sks=b&skv=2025-07-05&sig=pWcnlWAY/YxUDZo9FuofLiAe8avmL8/QWHWRcg1mJBY%3D>,
 'thumbnail': <Asset href=https://naipeuwest.blob.core.windows.net/naip/v002/ca/2022/ca_060cm_2022/34119/m_3411935_sw_11_060_20220513.200.jpg?st=2025-11-24T18%3A04%3A41Z&se=2025-11-25T18%3A49%3A41Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-11-23T16%3A24%3A35Z&ske=2025-11-30T16%3A24%3A35Z&sks=b&skv=2025-07-05&sig=pWcnlWAY/YxUDZo9FuofLiAe8avmL8/QWHWRcg1mJBY%3D>,
 'tilejson': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=naip&item=ca_m_3411

In [18]:
# More information about each asset in the item
for key in item.assets.keys():
    print(key, '-', item.assets[key].title)

image - RGBIR COG tile
thumbnail - Thumbnail
tilejson - TileJSON with default rendering
rendered_preview - Rendered preview


In [19]:
# PLot rendered preview
# Access URL via href attribute
Image(url=item.assets['rendered_preview'].href)

## Load data
The raster data in our current `item` is in the large `image` asset

In [ ]:
# Open NAIP scene over Santa Barbara
sb = rioxr.open_rasterio(item.assets['image'].href)
sb

<xarray.DataArray (band: 4, y: 12480, x: 10550)> Size: 527MB
[526656000 values with dtype=uint8]
Coordinates:
  * band         (band) int32 16B 1 2 3 4
  * x            (x) float64 84kB 2.469e+05 2.469e+05 ... 2.533e+05 2.533e+05
  * y            (y) float64 100kB 3.814e+06 3.814e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int32 4B 0
Attributes:
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       1
    TIFFTAG_YRESOLUTION:       1
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    AREA_OR_POINT:             Area
    scale_factor:              1.0
    add_offset:                0.0

# Do not actually run the code below

In [25]:
# Plot raster with correct ratio
size = 6 
aspect = sb.rio.width / sb.rio.height

# Select RGB bands 
sb.sel(band=[1,2,3]).plot.imshow(size=size, aspect=aspect)

KeyboardInterrupt: 

# Copernicus Digital Elevation model

In [ ]:
# Search for item in catalog
cop_dem = catalog.get_child('cop-dem-glo-90')
cop_dem

<CollectionClient id=cop-dem-glo-90>

In [41]:
# Catalog search for Cop DEM
search = catalog.search(
    collections = ['cop-dem-glo-90'], 
                   intersects = bbox)
search

In [ ]:
# Intermediate step- ask for the item
items = search.item_collection()
items

In [ ]:
item = items[0]
item.assets

{'data': <Asset href=https://elevationeuwest.blob.core.windows.net/copernicus-dem/COP90_hh/Copernicus_DSM_COG_30_N34_00_W120_00_DEM.tif?st=2025-11-24T18%3A26%3A44Z&se=2025-11-25T19%3A11%3A44Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-11-25T16%3A34%3A03Z&ske=2025-12-02T16%3A34%3A03Z&sks=b&skv=2025-07-05&sig=CG/DPOGTgCGOnAnPGMpTcUP76pVerdYabjJn%2Bwm9TPA%3D>,
 'tilejson': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=cop-dem-glo-90&item=Copernicus_DSM_COG_30_N34_00_W120_00_DEM&assets=data&colormap_name=terrain&rescale=-1000%2C4000&format=png>,
 'rendered_preview': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=cop-dem-glo-90&item=Copernicus_DSM_COG_30_N34_00_W120_00_DEM&assets=data&colormap_name=terrain&rescale=-1000%2C4000&format=png>}

In [46]:
item.properties

{'gsd': 90,
 'datetime': '2021-04-22T00:00:00Z',
 'platform': 'TanDEM-X',
 'proj:shape': [1200, 1200],
 'proj:transform': [0.0008333333333333334,
  0.0,
  -120.00041666666667,
  0.0,
  -0.0008333333333333334,
  35.000416666666666],
 'proj:code': 'EPSG:4326'}

In [44]:
# Open DEM of Santa Barbara
dem = rioxr.open_rasterio(item.assets['data'].href)
dem

<xarray.DataArray (band: 1, y: 1200, x: 1200)> Size: 6MB
[1440000 values with dtype=float32]
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 10kB -120.0 -120.0 -120.0 ... -119.0 -119.0 -119.0
  * y            (y) float64 10kB 35.0 35.0 35.0 35.0 ... 34.0 34.0 34.0 34.0
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:  Point
    scale_factor:   1.0
    add_offset:     0.0

In [47]:
dem.min().item()

-1.5

In [48]:
dem.max().item()

2689.8251953125

In [50]:
print(f"Minimum elevation: {dem.min().item()}, Maximum elevation: {dem.max().item():.2f}")

Minimum elevation: -1.5, Maximum elevation: 2689.83
